In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

In [6]:
full_df = pd.read_csv('chm_jail_data.csv', na_values = ['nan', 'Unknown', 'Other'])
selected_columns = [
    'booking_date', 'jacket_number','age_at_arrest', 'age_at_release', 'hours', 'marital_status', 'employment_status',
    'city', 'race', 'sex', 'STATE', 'citizenship', 'occupation',
    'school', 'superhighlevel', 'prisoner_type'
]

df = full_df[selected_columns]
df

/var/folders/ss/n8_jvt095flbv8pcsl15jyqw0000gn/T/ipykernel_48667/2763648481.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv('chm_jail_data.csv', na_values = ['nan', 'Unknown', 'Other'])


,booking_date,jacket_number,age_at_arrest,age_at_release,hours,marital_status,employment_status,city,race,sex,STATE,citizenship,occupation,school,superhighlevel,prisoner_type
0,1/1/2012,22914,51.0,51.0,1,Divorced,Employed - Full Time,CHAMPAIGN,White,Male,ILLINOIS,US,CONSTRUCTION WORKER,Graduated from high school,DUI,Misdemeanor Arraignment
1,1/1/2012,22914,51.0,51.0,1,Divorced,Employed - Full Time,CHAMPAIGN,White,Male,ILLINOIS,US,CONSTRUCTION WORKER,Graduated from high school,Traffic,Misdemeanor Arraignment
2,1/1/2012,22914,51.0,51.0,1,Divorced,Employed - Full Time,CHAMPAIGN,White,Male,ILLINOIS,US,CONSTRUCTION WORKER,Graduated from high school,Public Order,Misdemeanor Arraignment
3,1/1/2012,1024225,32.0,32.0,32,Single,Employed - Full Time,CHAMPAIGN,Hispanic,Male,ILLINOIS,US,UNEMPLOYED,Attends non-local school,Violent,Felony Arraignment
4,1/1/2012,1024225,32.0,32.0,32,Single,Employed - Full Time,CHAMPAIGN,Hispanic,Male,ILLINOIS,US,UNEMPLOYED,Attends non-local school,Violent,Felony Arraignment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67918,6/30/2018,1069755,32.0,32.0,0,Married,Employed - Full Time,URBANA,White,Female,ILLINOIS,US,"SERVICE PERSONNEL(HOTEL,RESTAURANT,NIGHT CLUB)",Non-attender,Traffic,Intermittent
67919,6/30/2018,1000385,30.0,30.0,1,Single,Employed - Full Time,CHAMPAGIN,Black,Female,ILLINOIS,US,"SERVICE PERSONNEL(HOTEL,RESTAURANT,NIGHT CLUB)",Graduated from high school,Property,Misdemeanor Arraignment
67920,6/30/2018,990152,29.0,29.0,1,Single,Unemployed,CHAMPAIGN,Black,Female,ILLINOIS,US,TEACHER,Graduated from high school,Public Order,Misdemeanor Arraignment
67921,6/30/2018,34365,46.0,46.0,13,Single,Employed - Full Time,RANTOUL,White,Female,ILLINOIS,US,"CLERKS(GASSTATIONATTENDANT,CONVIENCESTORECLERK)",Non-attender,Violent,Misdemeanor Arraignment


In [7]:
new_df = full_df.drop_duplicates(subset=['booking_date', 'jacket_number'])
full_df['prisoner_type'].unique()

array(['Misdemeanor Arraignment', 'Felony Arraignment',
       'Traffic Arraignment', 'Misdemeanor Sentenced Other',
       'Traffic Sentenced CCCC', 'Felony Sentenced IDOC',
       'Felony Sentenced CCCC', 'Traffic Pre-Trial', 'Felony Other',
       'Civil Pre-Trial', nan, 'Civil Other', 'Felony Pre-Trial',
       'Hold Other', 'Juvenile Misdemeanor Warrant Arrest',
       'Misdemeanor Other', 'Misdemeanor Sentenced CCCC',
       'Felony Pre-Sentence', 'Felony Sentenced Other', 'EHD',
       'Juvenile Felony Sentence CCJDC', 'Hold Sentenced IDOC',
       'Civil Pre-Sentence', 'Traffic Sentenced IDOC', 'Traffic Other',
       'Juvenile DOC Warrant Arrest', 'Misdemeanor Pre-Trial',
       'Remanded to DHS', 'Civil Sentenced CCCC',
       'Felony Presentence DUI', 'Juvenile DOC Commitment Champaign Co',
       'Juvenile Felony CO-temp Detention', 'Felony Pre-Trial DUI',
       'OV Pre-Trial', 'Petition To Revoke', 'Direct Criminal Contemp',
       'Misdemeanor Pre-Sentence', 'Intermitten

Counting Total NaN Values:

In [8]:
nan_counts_list = []

for column in selected_columns:
    nan_count = full_df[column].isna().sum()
    unique_count = full_df[column].nunique()
    nan_counts_list.append({'Column': column, 'NaN Count': nan_count, 'Unique Count': unique_count})

nan_counts = pd.DataFrame(nan_counts_list)
nan_counts

,Column,NaN Count,Unique Count
0,booking_date,0,2373
1,jacket_number,0,19614
2,age_at_arrest,462,70
3,age_at_release,462,70
4,hours,0,2934
5,marital_status,383,6
6,employment_status,857,7
7,city,274,1107
8,race,818,6
9,sex,462,2


## Dropping Rows with at least 5 separate NaN values present

In [9]:
clean_df = new_df.copy()

clean_df['r_count'] = new_df.groupby('jacket_number')['jacket_number'].transform('count') - 1
clean_df['r'] = clean_df['r_count'] > 0

In [10]:
clean_df.dropna(subset=['sex'], inplace=True)
nan_counts_list = []

for column in selected_columns:
    nan_count = clean_df[column].isna().sum()
    unique_count = clean_df[column].nunique()
    nan_counts_list.append({'Column': column, 'NaN Count': nan_count, 'Unique Count': unique_count})

nan_counts = pd.DataFrame(nan_counts_list)
nan_counts

,Column,NaN Count,Unique Count
0,booking_date,0,2373
1,jacket_number,0,19608
2,age_at_arrest,0,70
3,age_at_release,0,70
4,hours,0,2928
5,marital_status,217,6
6,employment_status,529,7
7,city,127,1107
8,race,214,6
9,sex,0,2


In [11]:
crime_mapping = {
    'DUI': 2,
    'Traffic': 0,
    'Public Order': 1,
    'Violent': 5,
    'Drug': 4,
    'Domestic Violence': 6,
    'Property': 3,
    'Sex': 7
}
clean_df['superhighlevel_no'] = df['superhighlevel'].map(crime_mapping)

sex_mapping = {'Male': 0, 'Female': 1}
clean_df['sex_no'] = clean_df['sex'].map(sex_mapping)

race_mapping = {'White': 0, 'Asian/Pacific Islander': 1, 'White (Hispanic)': 2, 'Native American': 3, 'Hispanic': 4, 'Black': 5, np.nan: 6}
clean_df['race_no'] = clean_df['race'].map(race_mapping)

In [12]:
school_level_mapping = {
    'Graduated from high school': 'High School',
    'Attends non-local school': 'High School',
    np.nan: np.nan,
    'Completed GED Program': 'High School',
    'READY High School': 'High School',
    'Non-attender': 'High School',
    'Urbana High School': 'High School',
    'Central High School': 'High School',
    'Centennial High School': 'High School',
    'Circle Academy': 'High School',
    'Parkland Community College': 'College',
    'Rantoul Township High School': 'High School',
    'Urbana Adult Ed': 'High School',
    'GED program--enrolled in or starting soon': 'High School',
    'St. Joseph-Ogden High School': 'High School',
    'Unity High School': 'High School',
    'The Pavilion School': 'High School',
    'Columbia Center High School': 'High School',
    'Attends other local school': 'High School',
    'Mahomet-Seymour High School': 'High School',
    'Storefront School': 'High School',
    'Unity Junior High School': 'Middle School',
    'READY Middle School': 'Middle School',
    'Academic Academy': 'High School',
    'St. Joseph Junior High School': 'Middle School',
    'Unknown to minor': np.nan,
    'Heritage Senior High School': 'High School',
    'Fisher Junior-Senior High School': 'High School',
    'HS of Saint Thomas Moore': 'High School',
    'Edison Middle School': 'Middle School',
    'University Laboratory High School': 'High School',
    'Heritage Junior High School': 'Middle School',
    'Urbana Middle School': 'Middle School',
    'Yankee Ridge': 'Elementary School',
    'Gerber School at CCH': 'High School',
    'Judah Christian': 'High School',
    'Novac Academy': 'High School',
    'Other': np.nan
}
clean_df['school'] = df['school'].map(school_level_mapping)

In [13]:
citizenship_mapping = {
    'US': 'UNITED STATES',
    'UNITED STATES': 'UNITED STATES',
    'Mexico': 'MEXICO',
    'Phillipines': 'PHILIPPINES',
    'Guatemala': 'GUATEMALA',
    'China (Republic of)': 'CHINA',
    'GAMBIA, THE': 'GAMBIA',
    'DENMARK': 'DENMARK',
    'Canada': 'CANADA',
    'Scotland': 'SCOTLAND',
    'CONGO, DEMOCRATIC REPUBLIC OF THE': 'CONGO',
    'South Korea': 'SOUTH KOREA',
    'Pakistan': 'PAKISTAN',
    'ROMANIA': 'ROMANIA',
    'ECUADOR': 'ECUADOR',
    'UKRAINE': 'UKRAINE',
    'Nigeria': 'NIGERIA',
    'India': 'INDIA',
    'ZAMBIA': 'ZAMBIA',
    'LEBANON': 'LEBANON',
    'Brazil': 'BRAZIL',
    'BOSNIA & HERZEGOVINA': 'BOSNIA AND HERZEGOVINA',
    'TAIWAN': 'TAIWAN',
    'ANGOLA': 'ANGOLA',
    'AUSTRALIA': 'AUSTRALIA',
    'United Kingdom': 'UNITED KINGDOM',
    'PERU': 'PERU',
    'Poland': 'POLAND',
    'Russia': 'RUSSIA',
    'ISRAEL': 'ISRAEL',
    'LIBERIA': 'LIBERIA',
    'CONGO (BRAZZAVILLE)': 'CONGO',
    'Iran': 'IRAN',
    'HONDURAS': 'HONDURAS',
    'GERMANY': 'GERMANY',
    'ARGENTINA': 'ARGENTINA',
    'COSTA RICA': 'COSTA RICA',
    'THAILAND': 'THAILAND',
    'France': 'FRANCE',
    'AUSTRIA': 'AUSTRIA',
    'JORDAN': 'JORDAN',
    'TURKEY, REPUBLIC OF': 'TURKEY',
    'NICARAGUA': 'NICARAGUA',
    'KENYA': 'KENYA',
    'KOREA, SOUTH': 'SOUTH KOREA',
    'BELGIUM': 'BELGIUM',
    'KAZAKHSTAN': 'KAZAKHSTAN',
    'NORWAY': 'NORWAY',
    'Cambodia': 'CAMBODIA',
    'Africa': 'AFRICA',
    'KOREA, NORTH': 'NORTH KOREA',
    'EL SALVADOR': 'EL SALVADOR',
    'SAUDIA ARABIA': 'SAUDI ARABIA',
    'NIGERIA': 'NIGERIA',
    'CHILE': 'CHILE',
    'CANADA': 'CANADA',
    'Laos': 'LAOS',
    'Jamaica': 'JAMAICA',
    'Vietnam': 'VIETNAM',
    'CAMBODIA': 'CAMBODIA',
    'HAITI': 'HAITI',
    'ALGERIA': 'ALGERIA',
    'CUBA': 'CUBA',
    'Portuguese': 'PORTUGAL',
    'Macedonia': 'MACEDONIA',
    'VIETNAM': 'VIETNAM',
    'BOLIVIA': 'BOLIVIA',
    'COLOMBIA': 'COLOMBIA',
    'DOMINICAN REPUBLIC': 'DOMINICAN REPUBLIC',
    'MOROCCO': 'MOROCCO',
    'ALBANIA': 'ALBANIA',
    'BELARUS': 'BELARUS',
    'LIBYA': 'LIBYA',
    'TONGA': 'TONGA',
    'SUDAN': 'SUDAN',
    'Ireland': 'IRELAND',
    'NEPAL': 'NEPAL',
    np.nan: np.nan
}
clean_df['citizenship'] = df['citizenship'].map(citizenship_mapping)

In [14]:
occupation_mapping = {
    'CONSTRUCTION WORKER': 'CONSTRUCTION WORKER',
    'UNEMPLOYED': 'UNEMPLOYED',
    'LABOR POOLS,lABORER,FRUIT PICKER,': 'LABORER',
    'PROFESSIONAL': 'PROFESSIONAL',
    'SERVICE PERSONNEL(HOTEL,RESTAURANT,NIGHT CLUB)': 'SERVICE INDUSTRY',
    'MECHANIC(REPAIR PERSON)': 'MECHANIC',
    'FACTORY WORKER': 'FACTORY WORKER',
    'RETAIL SALES,REAL ESTATE,INSURANCE,FREELANCE,': 'SALES',
    'RETIRED': 'RETIRED',
    'DISABLED': 'DISABLED',
    'ARMED SERVICES': 'MILITARY',
    'STUDENT(HIGH/MIDDLE/ELE./COLLEGE/VOCATIONAL)': 'STUDENT',
    'CLERKS(GASSTATIONATTENDANT,CONVIENCESTORECLERK)': 'CLERK',
    'HOUSEWIFE': 'HOUSEWIFE',
    'JANITORIAL': 'JANITORIAL',
    'LAWN WORKERS,LANDSCAPING': 'LANDSCAPER',
    'SELF EMPLOYED': 'SELF EMPLOYED',
    'ELECTRICIAN': 'ELECTRICIAN',
    'OFFICE WORKER': 'OFFICE WORKER',
    'FACTORY(MANAGEMENT,WORKER,ETC.)': 'FACTORY WORKER',
    'MASON,ROOFER,PAINTER,PLUMBER,': 'TRADESMAN',
    'TEACHER': 'TEACHER',
    'DAY CARE WORKER(BABYSITTER,ETC.)': 'CHILD CARE',
    'DRIVER(TAXI,BUS,TRUCK,LIMO,ETC.,)': 'DRIVER',
    'CARPENTER': 'CARPENTER',
    'FIREMAN': 'FIREFIGHTER',
    'MEDICAL - DOCTOR/DENTIST/ETC': 'MEDICAL PROFESSIONAL',
    'OFFICE(MANAGEMENT,WORKER,BOOKKEEPING,ACCOUNTANT,SE': 'OFFICE WORKER',
    'MEDICAL - NURSE/AIDE/ETC': 'MEDICAL PROFESSIONAL',
    'ENTERTAINER(DANCER,SINGER,COMEDIAN)': 'ENTERTAINER',
    'COSMOTOLIGIST(HARI,NAILS,FACIAL,ETC.)': 'COSMETOLOGIST',
    'PUBLIC SAFETY/LAW ENFORCEMENT - POLICE': 'LAW ENFORCEMENT',
    'BANK(TELLER,BANKER,MORTGAGEBROKER)': 'BANKING',
    'GUARD(SECURITY,ETC.)': 'SECURITY GUARD',
    'FACTORY MANAGEMENT': 'MANAGER',
    'OFFICE MANAGEMENT': 'MANAGER',
    'POSTAL EMPLOYEE': 'POSTAL WORKER',
    'MEDICAL - DOCTOR/DENTIST/NURSE/EMTS,': 'MEDICAL PROFESSIONAL',
    'ACTOR/MODEL/ACTRESS': 'ENTERTAINER',
    'POLICE (OFFICERS, SUPPORT PERSONNEL, ETC.)': 'LAW ENFORCEMENT',
    'PUBLIC EMPLOYEE': 'PUBLIC EMPLOYEE',
    'ATTORNEY': 'LAWYER',
    'CORRECTIONAL OFFICER': 'CORRECTIONAL OFFICER',
    'COOK': 'COOK',
    'PROFESSIONAL ATHLETE': 'ATHLETE',
    'PHOTOGRAPHER': 'PHOTOGRAPHER',
    'REPORTER,(NEWSPAPER, T.V.,)': 'JOURNALIST',
    'PROSTITUTION': 'PROSTITUTION',
    'AIRLINE PERSONNEL': 'AIRLINE WORKER',
    'PROFESSIONAL MUSICIAN': 'MUSICIAN',
    'FLORIST,ARTS/CRAFTS': 'FLORIST/ARTIST',
    'AMUSEMENT(DISNEY, SEAWORLD,MGM,WET&WILD,ETC.)': 'AMUSEMENT PARK WORKER',
    'EXTERMINATOR': 'EXTERMINATOR',
    'PACE OPERATOR': np.nan,
    'HONE': np.nan,
    'OTHER': np.nan,
    'ENCE': np.nan,
    'ANAG': np.nan,
    'HIER': np.nan,
    'OFER': np.nan,
    'NANC': np.nan,
    'NO VISIBLE MEANS': np.nan,
    'UNKNOWN': np.nan,
    np.nan: np.nan
}

clean_df['occupation'] = df['occupation'].map(occupation_mapping)


In [15]:
def correct_city(city):
    if isinstance(city, str) and city.lower().startswith('chi'):
        return 'CHICAGO'
    elif isinstance(city, str) and city.lower().startswith('cha'):
        return 'CHAMPAIGN'
    else:
        return city

clean_df['city'] = df['city'].apply(correct_city)

## Imputation Process Using MICE

### Overview

MICE (Multiple Imputation by Chained Equations) is a method for imputing missing data in a dataset.

### Methodology

1. **Data Preparation**: Clean and prepare the dataset, handling outliers and encoding categorical variables.

2. **MICE Imputation**: Apply MICE to impute missing values iteratively, based on observed values of other variables.

3. **Multiple Imputation**: Generate multiple imputed datasets using MICE.

4. **Analysis**: Analyze each imputed dataset separately, then combine results using Rubin's rules.

### Usage

- **Handling Missing Data**: MICE is versatile for handling missing data in datasets with numerical and categorical variables.

- **Modeling and Inference**: Imputed datasets can be used for regression, classification, and clustering, providing useful statistical inference and accurate modeling results.

### School

In [16]:
columns_to_clean = ['school', 'race', 'citizenship','occupation', 'superhighlevel', 'marital_status', 'employment_status', 'city','STATE', 'prisoner_type']

In [17]:
clean_df['school'].value_counts()

school
High School          37570
College                318
Middle School           26
Elementary School        3
Name: count, dtype: int64

In [18]:
impute_col = 'school'
impute_col_encoded = 'school_no'

le = preprocessing.LabelEncoder()

le.classes_ = np.array(['High School', 'College', 'Middle School', 'Elementary School', np.nan])
clean_df[impute_col_encoded] = le.transform(clean_df[impute_col].values)

In [19]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x == 4 else x)

# Initialize and fit the IterativeImputer
imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'sex_no', 'race_no', 'school_no']]
imputer.fit(df_train)

# Transform and round the imputed values
df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

# Map imputed values back to original occupation categories
reverse_school_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['school_imputed'] = clean_df[impute_col_encoded].map(reverse_school_mapping)

# Create a DataFrame to show what the NaN values were imputed as
nan_impute_data = []

nan_indices = clean_df[clean_df[impute_col].isna()].index

for index in nan_indices:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'school_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_school_impute_df = pd.DataFrame(nan_impute_data)
nan_school_impute_df['Imputed_Value'].value_counts()

Imputed_Value
High School    2341
Name: count, dtype: int64

## Race

In [20]:
impute_col = 'race'
impute_col_encoded = 'race_no'

le.classes_ = np.array(['Black', 'White', 'Hispanic', 'Asian/Pacific Islander',
       'Native American', 'White (Hispanic)', 'nan'])
clean_df[impute_col_encoded] = le.transform(clean_df[impute_col].values)

In [21]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x == 6 else x)

imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'superhighlevel_no', 'sex_no', 'race_no', 'school_no']]
imputer.fit(df_train)

df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

reverse_race_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['race_imputed'] = clean_df[impute_col_encoded].map(reverse_race_mapping)

nan_impute_data = []

nan_indices = clean_df[clean_df[impute_col].isna()].index

for index in nan_indices:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'race_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_race_impute_df = pd.DataFrame(nan_impute_data)
nan_race_impute_df['Imputed_Value'].value_counts()

Imputed_Value
Black    211
White      3
Name: count, dtype: int64

## Citizenship

In [22]:
clean_df['citizenship'].value_counts()

citizenship
UNITED STATES             38558
MEXICO                      717
GUATEMALA                   246
CHINA                        83
SOUTH KOREA                  71
                          ...  
NORTH KOREA                   1
BOSNIA AND HERZEGOVINA        1
KAZAKHSTAN                    1
PAKISTAN                      1
NEPAL                         1
Name: count, Length: 69, dtype: int64

In [23]:
impute_col = 'citizenship'
impute_col_encoded = 'citizenship_no'

le.fit(clean_df.loc[:, impute_col])
citizenship_value_counts = clean_df['citizenship'].value_counts()
sorted_countries = citizenship_value_counts.index.tolist()
sorted_countries = [country for country in sorted_countries if not pd.isnull(country)]
sorted_le_classes = sorted(le.classes_, key=lambda x: sorted_countries.index(x) if x in sorted_countries else len(sorted_countries))

le.classes_ = np.array(sorted_le_classes)

clean_df[impute_col_encoded] = le.transform(clean_df[impute_col].values)


In [24]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x == 69 else x)

imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'school_no', 'superhighlevel_no', 'sex_no', 'race_no', 'citizenship_no']]
imputer.fit(df_train)

df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

reverse_citizenship_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['citizenship_imputed'] = clean_df[impute_col_encoded].map(reverse_citizenship_mapping)

nan_impute_data = []

for index in clean_df[clean_df[impute_col].isna()].index:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'citizenship_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_citizenship_impute_df = pd.DataFrame(nan_impute_data)
nan_citizenship_impute_df['Imputed_Value'].value_counts()

Imputed_Value
UNITED STATES    287
Name: count, dtype: int64

## Prisoner Type

In [25]:
impute_col = 'prisoner_type'
impute_col_encoded = 'prisoner_type_no'

le.fit(clean_df.loc[:, impute_col])
prisoner_type_value_counts = clean_df['prisoner_type'].value_counts()
sorted_prisoner_types = prisoner_type_value_counts.index.tolist()
sorted_prisoner_types = [ptype for ptype in sorted_prisoner_types if not pd.isnull(ptype)]
sorted_le_classes = sorted(le.classes_, key=lambda x: sorted_prisoner_types.index(x) if x in sorted_prisoner_types else len(sorted_prisoner_types))
le.classes_ = np.array(sorted_le_classes)

clean_df[impute_col_encoded] = le.transform(clean_df[impute_col].values)


In [26]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x >= 50 else x)

imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'sex_no', 'race_no', 'school_no', 'citizenship_no', impute_col_encoded]]
imputer.fit(df_train)

df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

reverse_superhighlevel_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['prisoner_type_imputed'] = clean_df[impute_col_encoded].map(reverse_superhighlevel_mapping)

nan_impute_data = []

nan_indices = clean_df[clean_df[impute_col].isna()].index

for index in nan_indices:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'prisoner_type_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_superhighlevel_impute_df = pd.DataFrame(nan_impute_data)
nan_superhighlevel_impute_df['Imputed_Value'].value_counts()


Imputed_Value
Felony Sentenced IDOC     243
EHD                        13
Misdemeanor Pre-Trial       5
Misdemeanor Other           4
Felony Pre-Trial            1
Traffic Sentenced CCCC      1
OV Pre-Trial                1
Name: count, dtype: int64

## Occupation

In [27]:
impute_col = 'occupation'
impute_col_encoded = 'occupation_no'

le.fit(clean_df.loc[:, impute_col])

occupation_value_counts = clean_df['occupation'].value_counts()
sorted_occupations = occupation_value_counts.index.tolist()
sorted_occupations = [occupation for occupation in sorted_occupations if not pd.isnull(occupation)]
sorted_le_classes = sorted(le.classes_, key=lambda x: sorted_occupations.index(x) if x in sorted_occupations else len(sorted_occupations))
le.classes_ = np.array(sorted_le_classes)

clean_df[impute_col_encoded] = le.transform(clean_df[impute_col].values)
le.classes_

array(['UNEMPLOYED', 'SERVICE INDUSTRY', 'FACTORY WORKER', 'STUDENT',
       'CONSTRUCTION WORKER', 'SALES', 'SELF EMPLOYED', 'DISABLED',
       'LABORER', 'CLERK', 'MEDICAL PROFESSIONAL', 'MECHANIC',
       'JANITORIAL', 'OFFICE WORKER', 'PROFESSIONAL', 'LANDSCAPER',
       'RETIRED', 'DRIVER', 'TRADESMAN', 'CARPENTER', 'TEACHER',
       'MANAGER', 'ELECTRICIAN', 'CHILD CARE', 'COSMETOLOGIST', 'BANKING',
       'POSTAL WORKER', 'ENTERTAINER', 'SECURITY GUARD', 'MILITARY',
       'PUBLIC EMPLOYEE', 'PHOTOGRAPHER', 'LAWYER', 'FIREFIGHTER',
       'LAW ENFORCEMENT', 'HOUSEWIFE', 'CORRECTIONAL OFFICER',
       'PROSTITUTION', 'AIRLINE WORKER', 'COOK', 'ATHLETE', 'JOURNALIST',
       'MUSICIAN', 'FLORIST/ARTIST', 'AMUSEMENT PARK WORKER', 'nan'],
      dtype='<U32')

In [28]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x >= 45 else x)

imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'prisoner_type_no','school_no', 'sex_no', 'race_no', 'citizenship_no', impute_col_encoded]]
imputer.fit(df_train)

df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

reverse_occupation_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['occupation_imputed'] = clean_df[impute_col_encoded].map(reverse_occupation_mapping)

nan_impute_data = []

nan_indices = clean_df[clean_df[impute_col].isna()].index

for index in nan_indices:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'occupation_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_occupation_impute_df = pd.DataFrame(nan_impute_data)
nan_occupation_impute_df['Imputed_Value'].value_counts()

Imputed_Value
STUDENT                1457
FACTORY WORKER         1041
CONSTRUCTION WORKER     638
SALES                   317
SELF EMPLOYED            54
SERVICE INDUSTRY         37
DISABLED                  8
UNEMPLOYED                1
Name: count, dtype: int64

## City

In [29]:
impute_col = 'city'
impute_col_encoded = 'city_no'

le.fit(clean_df.loc[:, impute_col])
city_value_counts = clean_df['city'].value_counts()
sorted_cities = city_value_counts.index.tolist()
sorted_cities = [city for city in sorted_cities if not pd.isnull(city)]
sorted_le_classes = sorted(le.classes_, key=lambda x: sorted_cities.index(x) if x in sorted_cities else len(sorted_cities))
le.classes_ = np.array(sorted_le_classes)

clean_df[impute_col_encoded] = le.transform(clean_df[impute_col])

In [30]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x >= 1068 else x)

imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'prisoner_type_no', 'sex_no']]
imputer.fit(df_train)

df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

reverse_city_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['city_imputed'] = clean_df[impute_col_encoded].map(reverse_city_mapping)

nan_impute_data = []

nan_indices = clean_df[clean_df[impute_col].isna()].index

for index in nan_indices:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'city_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_city_impute_df = pd.DataFrame(nan_impute_data)
nan_city_impute_df['Imputed_Value'].value_counts()


Imputed_Value
CHAMPAIGN    120
URBANA         7
Name: count, dtype: int64

## State

In [31]:
impute_col = 'STATE'
impute_col_encoded = 'STATE_no'

le.fit(clean_df.loc[:, impute_col])
state_value_counts = clean_df['STATE'].value_counts()
sorted_states = state_value_counts.index.tolist()
sorted_states = [state for state in sorted_states if not pd.isnull(state)]
sorted_le_classes = sorted(le.classes_, key=lambda x: sorted_states.index(x) if x in sorted_states else len(sorted_states))
le.classes_ = np.array(sorted_le_classes)

clean_df[impute_col_encoded] = le.transform(clean_df[impute_col])

In [32]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x >= 45 else x)

imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'superhighlevel_no', 'sex_no', 'race_no', 'school_no', 'citizenship_no', impute_col_encoded]]
imputer.fit(df_train)

df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

reverse_state_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['STATE_imputed'] = clean_df[impute_col_encoded].map(reverse_state_mapping)

nan_impute_data = []

nan_indices = clean_df[clean_df[impute_col].isna()].index

for index in nan_indices:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'STATE_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_state_impute_df = pd.DataFrame(nan_impute_data)
nan_state_impute_df['Imputed_Value'].value_counts()


Imputed_Value
ILLINOIS    147
INDIANA       1
Name: count, dtype: int64

## Marital Status

In [33]:
impute_col = 'marital_status'
impute_col_encoded = 'marital_status_no'

le.classes_ = np.array(['Single', 'Married', 'Divorced', 'Seperated', 'Significant Other',
       'Widowed', 'nan'])

clean_df[impute_col_encoded] = le.transform(clean_df[impute_col])

In [34]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x >= 6 else x)

imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'superhighlevel_no', 'sex_no', 'race_no', 'school_no', 'citizenship_no', 'marital_status_no']]
imputer.fit(df_train)

df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

reverse_marital_status_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['marital_status_imputed'] = clean_df[impute_col_encoded].map(reverse_marital_status_mapping)

nan_impute_data = []

nan_indices = clean_df[clean_df[impute_col].isna()].index

for index in nan_indices:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'marital_status_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_marital_status_impute_df = pd.DataFrame(nan_impute_data)
nan_marital_status_impute_df['Imputed_Value'].value_counts()

Imputed_Value
Single      163
Married      52
Divorced      2
Name: count, dtype: int64

## Employment Status

In [35]:
impute_col = 'employment_status'
impute_col_encoded = 'employment_status_no'

le.classes_ = np.array(['Unemployed', 'Employed - Full Time', 'Employed - Part Time',
       'Student', 'Self Employed', 'Retired', 'Laid Off', 'nan'])

clean_df[impute_col_encoded] = le.transform(clean_df[impute_col])

In [36]:
clean_df[impute_col_encoded] = clean_df[impute_col_encoded].map(lambda x: np.nan if x >= 7 else x)

imputer = IterativeImputer(random_state=42)
df_train = clean_df.loc[:, ['age_at_arrest', 'age_at_release', 'hours', 'superhighlevel_no', 'sex_no', 'race_no', 'school_no', 'citizenship_no', 'employment_status_no']]
imputer.fit(df_train)

df_imputed = imputer.transform(df_train)
clean_df[impute_col_encoded] = df_imputed[:, -1].round().astype(int)

reverse_employment_status_mapping = {i: label for i, label in enumerate(le.classes_)}
clean_df['employment_status_imputed'] = clean_df[impute_col_encoded].map(reverse_employment_status_mapping)

nan_impute_data = []

nan_indices = clean_df[clean_df[impute_col].isna()].index

for index in nan_indices:
    original_nan_value = np.nan
    imputed_value = clean_df.loc[index, 'employment_status_imputed']
    nan_impute_data.append({'Original_NaN': original_nan_value, 'Imputed_Value': imputed_value})

nan_employment_status_impute_df = pd.DataFrame(nan_impute_data)
nan_employment_status_impute_df['Imputed_Value'].value_counts()


Imputed_Value
Employed - Full Time    525
Unemployed                3
Employed - Part Time      1
Name: count, dtype: int64

# Creating Final DataFrame

In [37]:
final_columns = ['booking_date', 'jacket_number','age_at_arrest', 'age_at_release', 'hours', 'marital_status_imputed', 'employment_status_imputed',
    'city_imputed', 'race_imputed', 'sex', 'STATE_imputed', 'citizenship_imputed', 'occupation_imputed',
    'school_imputed', 'prisoner_type_imputed', 'marital_status_no', 'employment_status_no',
    'city_no', 'race_no', 'sex_no', 'STATE_no', 'citizenship_no',
    'occupation_no', 'school_no', 'prisoner_type_no', 'r', 'r_count']

final_df = clean_df[final_columns].copy()

column_mapping = {
    'marital_status_imputed': 'marital_status',
    'employment_status_imputed': 'employment_status',
    'city_imputed': 'city',
    'race_imputed': 'race',
    'STATE_imputed': 'STATE',
    'citizenship_imputed': 'citizenship',
    'occupation_imputed': 'occupation',
    'school_imputed': 'school',
    'prisoner_type_imputed': 'prisoner_type'
}
final_df.rename(columns=column_mapping, inplace=True)
pd.set_option('display.max_rows', 60)
final_df

,booking_date,jacket_number,age_at_arrest,age_at_release,hours,marital_status,employment_status,city,race,sex,...,city_no,race_no,sex_no,STATE_no,citizenship_no,occupation_no,school_no,prisoner_type_no,r,r_count
0,1/1/2012,22914,51.0,51.0,1,Divorced,Employed - Full Time,CHAMPAIGN,Black,Male,...,0,0,0,0,0,4,0,0,True,1
3,1/1/2012,1024225,32.0,32.0,32,Single,Employed - Full Time,CHAMPAIGN,Black,Male,...,0,0,0,0,0,0,0,2,True,4
5,1/1/2012,1024226,20.0,20.0,12,Single,Employed - Part Time,CHAMPAIGN,Black,Male,...,0,0,0,0,0,8,0,0,False,0
6,1/1/2012,1024227,20.0,20.0,10,Single,Unemployed,CHAMPAIGN,Black,Male,...,0,0,0,0,0,2,0,2,False,0
7,1/1/2012,1024228,17.0,17.0,10,Single,Unemployed,CHAMPAIGN,Black,Male,...,0,0,0,0,0,0,0,3,True,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67918,6/30/2018,1069755,32.0,32.0,0,Married,Employed - Full Time,URBANA,Black,Female,...,1,0,1,0,0,1,0,47,False,0
67919,6/30/2018,1000385,30.0,30.0,1,Single,Employed - Full Time,URBANA,Black,Female,...,1,0,1,0,0,1,0,0,False,0
67920,6/30/2018,990152,29.0,29.0,1,Single,Unemployed,URBANA,Black,Female,...,1,0,1,0,0,20,0,0,True,1
67921,6/30/2018,34365,46.0,46.0,13,Single,Employed - Full Time,URBANA,Black,Female,...,1,0,1,0,0,9,0,0,True,1


In [38]:
final_df.dropna(subset=['occupation'], inplace=True)
final_df.isna()

,booking_date,jacket_number,age_at_arrest,age_at_release,hours,marital_status,employment_status,city,race,sex,...,city_no,race_no,sex_no,STATE_no,citizenship_no,occupation_no,school_no,prisoner_type_no,r,r_count
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67918,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
67919,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
67920,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
67921,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [39]:
final_df.isna().sum()

booking_date            0
jacket_number           0
age_at_arrest           0
age_at_release          0
hours                   0
marital_status          0
employment_status       0
city                    0
race                    0
sex                     0
STATE                   0
citizenship             0
occupation              0
school                  0
prisoner_type           0
marital_status_no       0
employment_status_no    0
city_no                 0
race_no                 0
sex_no                  0
STATE_no                0
citizenship_no          0
occupation_no           0
school_no               0
prisoner_type_no        0
r                       0
r_count                 0
dtype: int64

In [40]:
final_df.to_csv('CLEAN_chm_jail_data.csv', index=False)

In [41]:
x = pd.read_csv("CLEAN_chm_jail_data.csv")
x.isna().sum()

booking_date            0
jacket_number           0
age_at_arrest           0
age_at_release          0
hours                   0
marital_status          0
employment_status       0
city                    0
race                    0
sex                     0
STATE                   0
citizenship             0
occupation              0
school                  0
prisoner_type           0
marital_status_no       0
employment_status_no    0
city_no                 0
race_no                 0
sex_no                  0
STATE_no                0
citizenship_no          0
occupation_no           0
school_no               0
prisoner_type_no        0
r                       0
r_count                 0
dtype: int64